In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("C:\\Users\\arups\\OneDrive\\Desktop\\COLLECTION\\PROGRAMS\\TEST_CSVs\\titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(['PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1 , inplace = True)

In [4]:
# Step-1 : train_test_split

X_train , X_test , y_train , y_test = train_test_split(df.drop('Survived' , axis=1) , df['Survived'] , random_state=42 , test_size = 0.2)

In [5]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [10]:
# imputation transformer

trf1 = ColumnTransformer([
    ('impute_age' , SimpleImputer() , [2]),
    ('impute_embarked' , SimpleImputer(strategy='most_frequent') , [6])
    
] , remainder='passthrough')
# While using pipeline it's better use column index rather than column name while u applying imputer

In [9]:
# one hot encoding

trf2 = ColumnTransformer([
    
    ('impute_sex_embarked' , OneHotEncoder(sparse_output=False , handle_unknown='ignore') , [1,6])
    
] , remainder='passthrough')

In [8]:
# scaling

trf3 = ColumnTransformer([
    
    ('scale' , MinMaxScaler() , slice(0,10)) # After all transformation as we don't drop any column total number of col have 10
    
])

In [11]:
# feature selection

trf4 = SelectKBest(score_func=chi2 , k=8) # Out of 10 cols we use top 8 column

In [12]:
# train the model

trf5 = DecisionTreeClassifier()

# Create Pipeline

In [13]:
pipe = Pipeline([
    ('trf1' , trf1),
    ('trf2' , trf2),
    ('trf3' , trf3),
    ('trf4' , trf4),
    ('trf5' , trf5)
])

# Pipeline VS make_pipeline

pipeline requires naming of steps,make_pipeline does not
(same applies to ColumnTransformer VS make_column_transformer)

In [ ]:
# Alternate systex
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [14]:
# train
# If u don't do any model training then u call fit_transform() because here u do data preprocessing
# If u do model traininng the u call fit() so then in future u call predict()

pipe.fit(X_train , y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002BD55D68040>)),
                ('trf5', DecisionTreeClassifier())])

# Explore the pipeline

In [18]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_ # mean

array([29.49884615])

In [22]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [23]:
# Display pipeline
from sklearn import set_config
set_config(display='diagram')

In [24]:
# predict
y_pred = pipe.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

0.6256983240223464

# Cross Validation using Pipeline

In [26]:
# cross validation using cross_val_score

from sklearn.model_selection import cross_val_score
cross_val_score(pipe , X_train , y_train , cv=5 , scoring = 'accuracy').mean()

0.6391214419383433